In [5]:
!pip install -q playwright nest_asyncio
!playwright install chromium

import nest_asyncio
nest_asyncio.apply()


In [7]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
from bs4 import BeautifulSoup

async def scrape_case_data(cnr_number):
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()

            print("🔃 Loading page...")
            await page.goto("https://services.ecourts.gov.in/ecourtindia_v6/")

            print("✅ Page loaded. Clicking CNR tab...")
            await page.click("text='CNR Number'")

            print("⏳ Waiting for input field...")
            await page.wait_for_selector("#cnrno", timeout=10000)  # wait for input

            print("⌨️ Entering CNR number...")
            await page.fill("#cnrno", cnr_number)

            print("🔍 Submitting form...")
            await page.click("#searchbtn")

            print("⏳ Waiting for data to load...")
            await page.wait_for_timeout(4000)

            html = await page.content()
            await browser.close()
            return html

    except PlaywrightTimeoutError as e:
        print("❌ Timeout: Element not found.")
        return None
    except Exception as e:
        print("❌ Error occurred:", e)
        return None

# 🔧 Run the function
cnr = "HRFB010009602022"
html = await scrape_case_data(cnr)

if html:
    soup = BeautifulSoup(html, "html.parser")
    tables = soup.find_all("table")

    for i, table in enumerate(tables):
        print(f"\n--- Table {i+1} ---")
        print(table.text.strip())
else:
    print("⚠️ Scraping failed.")


🔃 Loading page...
✅ Page loaded. Clicking CNR tab...
⏳ Waiting for input field...
❌ Timeout: Element not found.
⚠️ Scraping failed.


In [15]:
import requests
from bs4 import BeautifulSoup
import sqlite3

# --- Input
cnr = "HRFB010009602022"
url = "https://services.ecourts.gov.in/ecourtindia_v6/cases/cnr_case_details.php"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Referer": "https://services.ecourts.gov.in/ecourtindia_v6/"
}

# --- Fetch
response = requests.post(url, headers=headers, data={"cnrno": cnr})
soup = BeautifulSoup(response.text, "html.parser")

# --- Flexible extractor (line-based)
def extract_by_keyword(soup, keyword):
    tags_to_check = ['div', 'td', 'span', 'p']
    for tag in tags_to_check:
        for el in soup.find_all(tag):
            text = el.get_text(strip=True)
            if keyword.lower() in text.lower() and ":" in text:
                key_val = text.split(":", 1)
                if len(key_val) == 2:
                    return key_val[1].strip()
    return "N/A"

# --- Extract fields
case_type = extract_by_keyword(soup, "case type")
filing_number = extract_by_keyword(soup, "filing number")
filing_date = extract_by_keyword(soup, "filing date")
registration_number = extract_by_keyword(soup, "registration number")
registration_date = extract_by_keyword(soup, "registration date")
status = extract_by_keyword(soup, "case status")
disposal_nature = extract_by_keyword(soup, "disposal")
decision_date = extract_by_keyword(soup, "decision date")
judge = extract_by_keyword(soup, "judge")

# --- Print results
print("📄 Case Type:", case_type)
print("📝 Filing Number:", filing_number)
print("📅 Filing Date:", filing_date)
print("📄 Registration Number:", registration_number)
print("📅 Registration Date:", registration_date)
print("⚖️ Status:", status)
print("✅ Nature of Disposal:", disposal_nature)
print("📅 Decision Date:", decision_date)
print("👨‍⚖️ Judge:", judge)

# --- Optional: Save to DB
conn = sqlite3.connect('court_cases.db')
c = conn.cursor()

c.execute('''
CREATE TABLE IF NOT EXISTS cases (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    cnr TEXT,
    case_type TEXT,
    filing_number TEXT,
    filing_date TEXT,
    registration_number TEXT,
    registration_date TEXT,
    status TEXT,
    disposal_nature TEXT,
    decision_date TEXT,
    judge TEXT,
    raw_html TEXT
)
''')

c.execute('''
INSERT INTO cases (
    cnr, case_type, filing_number, filing_date, registration_number,
    registration_date, status, disposal_nature, decision_date, judge, raw_html
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', (
    cnr, case_type, filing_number, filing_date, registration_number,
    registration_date, status, disposal_nature, decision_date, judge, response.text
))

conn.commit()
conn.close()

print("\n✅ Case data saved to 'court_cases.db'")


📄 Case Type: N/A
📝 Filing Number: N/A
📅 Filing Date: N/A
📄 Registration Number: N/A
📅 Registration Date: N/A
⚖️ Status: N/A
✅ Nature of Disposal: N/A
📅 Decision Date: N/A
👨‍⚖️ Judge: N/A

✅ Case data saved to 'court_cases.db'


In [16]:
!pip install flask-ngrok
